In [1]:
#!pip install 

In [1]:
from transformers import VitsModel, AutoTokenizer, AutoModelForSeq2SeqLM
from torch.quantization import  quantize_dynamic
import torch.nn as nn
import torch
from IPython.display import Audio
import os
import time
import pandas as pd

/workspace/.pyenv_mirror/user/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(f"current working dir ->{os.getcwd()}")
#os.chdir('..')
print(f"current working dir ->{os.getcwd()}")

current working dir ->/workspace/Cohort8-Ransom-Kuti-Ladipo/yb2audio/notebooks
current working dir ->/workspace/Cohort8-Ransom-Kuti-Ladipo/yb2audio


In [3]:
mms_model_name = "facebook/mms-tts-yor"
mms_model = VitsModel.from_pretrained(mms_model_name)
mms_tokenizer = AutoTokenizer.from_pretrained(mms_model_name)
nllb_model_name = 'facebook/nllb-200-distilled-600M'
youruba_lang = 'yor_Latn'
nllb_tokenizer = AutoTokenizer.from_pretrained(nllb_model_name)
nllb_model = AutoModelForSeq2SeqLM.from_pretrained(nllb_model_name)

Some weights of the model checkpoint at facebook/mms-tts-yor were not used when initializing VitsModel: ['posterior_encoder.wavenet.res_skip_layers.8.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'posterior_encoder.wavenet.in_layers.8.weight_g', 'posterior_encoder.wavenet.in_layers.14.weight_g', 'flow.flows.3.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'posterior_encoder.wavenet.in_layers.0.weight_g', 'posterior_encoder.wavenet.in_layers.7.weight_g', 'flow.flows.3.wavenet.in_layers.2.weight_g', 'posterior_encoder.wavenet.res_skip_layers.1.weight_v', 'flow.flows.1.wavenet.res_skip_layers.2.weight_g', 'posterior_encoder.wavenet.in_layers.10.weight_g', 'posterior_encoder.wavenet.in_layers.8.weight_v', 'flow.flows.3.wavenet.res_skip_layers.0.weight_g', 'flow.flows.3.wavenet.in_layers.0.weight_g', 'posterior_encoder.wavenet.res_skip_layers.6.weight_g', 'flow.flows.2.wavenet.res_skip_layers.0.weight_g', 'posterior_encoder.wavenet.res_skip_layer

In [4]:
def MMS_model_infer(text):
    inputs = mms_tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        output = mms_model(**inputs).waveform
    return Audio(output, rate=mms_model.config.sampling_rate)

def NLLB_infer(eng_text):
    inputs = nllb_tokenizer(eng_text, return_tensors='pt')
    translate_token = nllb_model.generate(**inputs, forced_bos_token_id = nllb_tokenizer.lang_code_to_id[youruba_lang],
                                          max_length = 50)
    outputs = nllb_tokenizer.batch_decode(translate_token, skip_special_tokens=True)[0]
    return outputs

def get_directory_size(directory_path):
    total_size = 0
    with os.scandir(directory_path) as entries:
        for entry in entries:
            if entry.is_file():
                total_size += entry.stat().st_size
            elif entry.is_dir():
                total_size += get_directory_size(entry.path)
    return round(total_size / (1024 * 1024), 2)

In [5]:
def inference(sample_text):
    time_1 = time.time()
    yor_text= NLLB_infer(sample_text)
    print(yor_text)
    yor_audio_out = MMS_model_infer(yor_text)
    time_2 = time.time()
    print(time_2 - time_1)
    return yor_audio_out
ans = inference("I will go to work,on sunday,for just 5000 naira, Thank God")
ans

Mo máa lọ síbi iṣẹ́ lọ́jọ́ Sunday, fún ẹ̀ẹ́dẹ́gbẹ̀rún márùn-ún naira, mo dúpẹ́ lọ́wọ́ ọlọ́run
5.7738282680511475


In [6]:
dev_df = pd.read_csv('data/raw/iroyinspeech/train.tsv', delimiter= '\t')
dev_df.head(2)

,client_id,path,sentence,up_votes,down_votes,age,gender,accents,variant,locale,segment
0,7bc2ed85985f28af0f62be69494e94d712557f6d7893a0...,common_voice_yo_36840058.mp3,Ọmọ ẹgbẹ́ òkùnkùn dèrò àtìmọ́lé torí nílùú Ìbà...,2,0,NaN,NaN,NaN,NaN,yo,NaN
1,7bc2ed85985f28af0f62be69494e94d712557f6d7893a0...,common_voice_yo_36840059.mp3,Ìyàwó àwọn ọlọ́pàá tó kú lásìkò ìwọ́de tó kọ...,2,0,NaN,NaN,NaN,NaN,yo,NaN


In [7]:
samples = dev_df[['path','sentence']][:50]
samples.head()

,path,sentence
0,common_voice_yo_36840058.mp3,Ọmọ ẹgbẹ́ òkùnkùn dèrò àtìmọ́lé torí nílùú Ìbà...
1,common_voice_yo_36840059.mp3,Ìyàwó àwọn ọlọ́pàá tó kú lásìkò ìwọ́de tó kọ...
2,common_voice_yo_36840060.mp3,Nítori ọrọ táa sọ lọ́jọ́sí ni Túndé yoo f...
3,common_voice_yo_36840061.mp3,Ilẹ̀ mímìtìtì ní ìlú ṣakí ti sọ àwọn olug...
4,common_voice_yo_36840342.mp3,"Ìgbòho ní Aláàfin, Ọ̀ọ̀nì, Olúbàdàn tì òun lẹ́..."


In [9]:
test = 27
print(samples['sentence'][test])
MMS_model_infer(samples['sentence'][test])

Iṣẹ́ àgbẹ̀ ṣe kókó lásìkò yìí.


In [10]:
audio_path = 'data/raw/iroyinspeech/clips/' + samples['path'][test]
Audio(audio_path, rate = mms_model.config.sampling_rate)

In [11]:
mms_model.config.sampling_rate

16000

Quantizattion of MMS

In [12]:
mms_path = 'development/MMS'
mms_model.save_pretrained(mms_path)
mms_tokenizer.save_pretrained(mms_path)

('development/MMS/tokenizer_config.json',
 'development/MMS/special_tokens_map.json',
 'development/MMS/vocab.json',
 'development/MMS/added_tokens.json')

In [13]:
get_directory_size(mms_path)

138.66

In [15]:
def MMS_Qmodel_infer(text):
    inputs = mms_tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        output = model_quant(**inputs).waveform
    return Audio(output, rate=mms_model.config.sampling_rate)

In [29]:
mms_quantized_model = quantize_dynamic(mms_model, qconfig_spec={torch.nn.Linear: torch.quantization.default_dynamic_qconfig}, dtype=torch.qint8)

In [30]:
for name, param in  mms_quantized_model.named_parameters():
    print(f"{name}: {param.dtype}")

text_encoder.embed_tokens.weight: torch.float32
text_encoder.encoder.layers.0.attention.emb_rel_k: torch.float32
text_encoder.encoder.layers.0.attention.emb_rel_v: torch.float32
text_encoder.encoder.layers.0.layer_norm.weight: torch.float32
text_encoder.encoder.layers.0.layer_norm.bias: torch.float32
text_encoder.encoder.layers.0.feed_forward.conv_1.weight: torch.float32
text_encoder.encoder.layers.0.feed_forward.conv_1.bias: torch.float32
text_encoder.encoder.layers.0.feed_forward.conv_2.weight: torch.float32
text_encoder.encoder.layers.0.feed_forward.conv_2.bias: torch.float32
text_encoder.encoder.layers.0.final_layer_norm.weight: torch.float32
text_encoder.encoder.layers.0.final_layer_norm.bias: torch.float32
text_encoder.encoder.layers.1.attention.emb_rel_k: torch.float32
text_encoder.encoder.layers.1.attention.emb_rel_v: torch.float32
text_encoder.encoder.layers.1.layer_norm.weight: torch.float32
text_encoder.encoder.layers.1.layer_norm.bias: torch.float32
text_encoder.encoder.lay

Quantizattion of NLLB

In [ ]:
layer_p = 'encoder.layers.0.self_attn_layer_norm.weight

In [28]:
nllb_quantized_model = quantize_dynamic(nllb_model, qconfig_spec={torch.nn.Linear: torch.quantization.default_dynamic_qconfig}, dtype=torch.qint8)

In [32]:
for name, param in nllb_quantized_model.named_parameters():
    print(f"{name}, {param.dtype}")

model.shared.weight, torch.float32
model.encoder.layers.0.self_attn_layer_norm.weight, torch.float32
model.encoder.layers.0.self_attn_layer_norm.bias, torch.float32
model.encoder.layers.0.final_layer_norm.weight, torch.float32
model.encoder.layers.0.final_layer_norm.bias, torch.float32
model.encoder.layers.1.self_attn_layer_norm.weight, torch.float32
model.encoder.layers.1.self_attn_layer_norm.bias, torch.float32
model.encoder.layers.1.final_layer_norm.weight, torch.float32
model.encoder.layers.1.final_layer_norm.bias, torch.float32
model.encoder.layers.2.self_attn_layer_norm.weight, torch.float32
model.encoder.layers.2.self_attn_layer_norm.bias, torch.float32
model.encoder.layers.2.final_layer_norm.weight, torch.float32
model.encoder.layers.2.final_layer_norm.bias, torch.float32
model.encoder.layers.3.self_attn_layer_norm.weight, torch.float32
model.encoder.layers.3.self_attn_layer_norm.bias, torch.float32
model.encoder.layers.3.final_layer_norm.weight, torch.float32
model.encoder.lay